**Интерфейс для удобного общения с виртуальным помощником:**

In [5]:
import gradio as gr
import clickhouse_connect
from embedding import E5LargeEmbeddingFunction
from similarity import bm25_ensemble_with_crossenc_answer, prep_query, bm25_fewshot_with_cross_encoder
from pprint import pprint
import requests
import json
import pandas as pd
import gc


client = clickhouse_connect.get_client(host='y1jzidyt9q.us-east-2.aws.clickhouse.cloud', port=8443, username='default', password='_lQ_JWXYQD3ym')
emb_func = E5LargeEmbeddingFunction()

def message_llm(system_prompt, user_input, temperature=0.6):
    '''Function to message (test) our LLM'''

    try:
        url = "https://live-relaxed-oryx.ngrok-free.app/v1/chat/completions"

        data = {
          "messages": [
            { "role": "system", "content": system_prompt},
            { "role": "user", "content": user_input}
          ],
          "temperature": 0.6,
          # "max_tokens": -1,
          # "stream": False
        }

        headers = {
            "Content-Type": "application/json"
        }

        response = requests.post(url, headers=headers, data=json.dumps(data))

        if response.status_code == 200:
            return eval(response.text.replace("\n  ", ""))['choices'][0]['message']['content']
        else:
            print("Ошибка запроса:\n", response.status_code, response.text)
            return ""
            
    except Exception as e:
        
        print(f"Error {e}!")
        
rag_prompt = """
### CONTEXT ###
1) ИСТОЧНИК 1: 
- {type_of_source1}
- {link_to_source1}
- {batch_1}

2) ИСТОЧНИК 2: 
- {type_of_source2}
- {link_to_source2}
- {batch_2}

3) ИСТОЧНИК 3:
- {type_of_source3}
- {link_to_source3}
- {batch_3}

### INSTRUCTION ###
На основании приведенных данных категории «ТЕКСТ» дай исчерпывающий ответ на
приведенный ниже вопрос:
- {QUESTION}
"""

few_shot_data = pd.read_csv('qa_generation/finetune_df_final.csv', index_col=0)
few_shot_data

rag_prompt_fw_ex = """
### EXAMPLES ###

1) Пример 1:
КОНТЕКСТ: {context_1}
ВОПРОС: {question_1}
ОТВЕТ: {answer_1}

2) Пример 2:
КОНТЕКСТ: {context_2}
ВОПРОС: {question_2}
ОТВЕТ: {answer_2}

3) Пример 3:
КОНТЕКСТ: {context_3}
ВОПРОС: {question_3}
ОТВЕТ: {answer_3}

### CONTEXT ###

1) ИСТОЧНИК 1: 
- {type_of_source1}
- {link_to_source1}
- {batch_1}

2) ИСТОЧНИК 2: 
- {type_of_source2}
- {link_to_source2}
- {batch_2}

3) ИСТОЧНИК 3:
- {type_of_source3}
- {link_to_source3}
- {batch_3}

### INSTRUCTION ###

На основании приведенных данных категории «ТЕКСТ» дай исчерпывающий ответ на
приведенный ниже вопрос:
- {QUESTION}
"""

sc_prompt = """
### CONTEXT ###

1) МНЕНИЕ 1: 
- {answer1}

2) МНЕНИЕ 2: 
- {answer2}

3) МНЕНИЕ 3:
- {answer3}

### INSTRUCTION ###
На основании приведенного контекста, дай итоговый ответ клиенту на его вопрос по принципу большинства.
"""

system_prompt = """
### ROLE ###
Ты - дружелюбный ассистент Центрального банка Российской Федерации. 
Исчерпывающе отвечай клиентам на возникшие вопросы. Если из контекста 
невозможно дать ответ на вопрос, пиши «Я не могу ответить на этот вопрос»."""

def ask_chatbot(mode, question):
    gc.collect()
    try:
        formatted_rag_prompt = ""
        query = prep_query(question)
        if mode == "zero-shot":

            result = bm25_ensemble_with_crossenc_answer(query=query, client=client, emb_func=emb_func, bm25_n_results=15, cr_enc_n_results=3, limit_knn=100, knn_docs_window=1)

            type_of_source1, type_of_source2, type_of_source3  = '', '', ''
            link_to_source1, link_to_source2, link_to_source3 = '', '', ''
            batch_1, batch_2, batch_3 = '', '', ''


            for idx, batch in enumerate(start=1, iterable=result):
                if idx==1:
                    if batch[5]:
                        type_of_source1 = 'ТАБЛИЦА'
                    else:
                        type_of_source1 = 'ТЕКСТ'
                    link_to_source1 = batch[2]
                    batch_1 = batch[3]

                if idx==2:
                    if batch[5]:
                        type_of_source2 = 'ТАБЛИЦА'
                    else:
                        type_of_source2 = 'ТЕКСТ'
                    link_to_source2 = batch[2]
                    batch_2 = batch[3]

                if idx==3:
                    if batch[5]:
                        type_of_source3 = 'ТАБЛИЦА'
                    else:
                        type_of_source3 = 'ТЕКСТ'
                    link_to_source3 = batch[2]
                    batch_3 = batch[3]


            formatted_rag_prompt = rag_prompt.format(
                type_of_source1=type_of_source1,
                link_to_source1=link_to_source1,
                batch_1=batch_1,
                type_of_source2=type_of_source2,
                link_to_source2=link_to_source2,
                batch_2=batch_2,
                type_of_source3=type_of_source3,
                link_to_source3=link_to_source3,
                batch_3=batch_3,
                QUESTION=question
            )

            answer = message_llm(system_prompt, formatted_rag_prompt)

            return f"{answer}\nИсточники: \n{link_to_source1}, \n{link_to_source2}, \n{link_to_source3}"

        elif mode == "few-shot":
            result = bm25_ensemble_with_crossenc_answer(query=query, client=client, emb_func=emb_func, bm25_n_results=15, cr_enc_n_results=3, limit_knn=100, knn_docs_window=1)

            type_of_source1, type_of_source2, type_of_source3  = '', '', ''
            link_to_source1, link_to_source2, link_to_source3 = '', '', ''
            batch_1, batch_2, batch_3 = '', '', ''

            for idx, batch in enumerate(start=1, iterable=result):
                if idx==1:
                    if batch[5]:
                        type_of_source1 = 'ТАБЛИЦА'
                    else:
                        type_of_source1 = 'ТЕКСТ'
                    link_to_source1 = batch[2]
                    batch_1 = batch[3]

                if idx==2:
                    if batch[5]:
                        type_of_source2 = 'ТАБЛИЦА'
                    else:
                        type_of_source2 = 'ТЕКСТ'
                    link_to_source2 = batch[2]
                    batch_2 = batch[3]

                if idx==3:
                    if batch[5]:
                        type_of_source3 = 'ТАБЛИЦА'
                    else:
                        type_of_source3 = 'ТЕКСТ'
                    link_to_source3 = batch[2]
                    batch_3 = batch[3]

            few_shot_examples = bm25_fewshot_with_cross_encoder(bm25_n_results=100, cr_enc_n_results=3, df=few_shot_data, query=query)
            formatted_examples = '\n\n'.join([f"Пример {i+1}:\nКОНТЕКСТ: {ex['context']}\nВОПРОС: {ex['question']}\nОТВЕТ: {ex['answer']}" for i, ex in enumerate(few_shot_examples)])

            formatted_rag_prompt = rag_prompt_fw_ex.format(
                context_1=few_shot_examples[0]['context'],
                question_1=few_shot_examples[0]['question'],
                answer_1=few_shot_examples[0]['answer'],
                context_2=few_shot_examples[1]['context'],
                question_2=few_shot_examples[1]['question'], 
                answer_2=few_shot_examples[1]['answer'],
                context_3=few_shot_examples[2]['context'],
                question_3=few_shot_examples[2]['question'],  
                answer_3=few_shot_examples[2]['answer'],  
                type_of_source1=type_of_source1,
                link_to_source1=link_to_source1,
                batch_1=batch_1,
                type_of_source2=type_of_source2,
                link_to_source2=link_to_source2,
                batch_2=batch_2,
                type_of_source3=type_of_source3,
                link_to_source3=link_to_source3,
                batch_3=batch_3,
                QUESTION=question
            )
            answer = message_llm(system_prompt, formatted_rag_prompt)

            return f"{answer}\nИсточники: \n{link_to_source1}, \n{link_to_source2}, \n{link_to_source3}"

        elif mode == "self-consistency":
            result = bm25_ensemble_with_crossenc_answer(query=query, client=client, emb_func=emb_func, bm25_n_results=15, cr_enc_n_results=3, limit_knn=100, knn_docs_window=1)

            type_of_source1, type_of_source2, type_of_source3  = '', '', ''
            link_to_source1, link_to_source2, link_to_source3 = '', '', ''
            batch_1, batch_2, batch_3 = '', '', ''


            for idx, batch in enumerate(start=1, iterable=result):
                if idx==1:
                    if batch[5]:
                        type_of_source1 = 'ТАБЛИЦА'
                    else:
                        type_of_source1 = 'ТЕКСТ'
                    link_to_source1 = batch[2]
                    batch_1 = batch[3]

                if idx==2:
                    if batch[5]:
                        type_of_source2 = 'ТАБЛИЦА'
                    else:
                        type_of_source2 = 'ТЕКСТ'
                    link_to_source2 = batch[2]
                    batch_2 = batch[3]

                if idx==3:
                    if batch[5]:
                        type_of_source3 = 'ТАБЛИЦА'
                    else:
                        type_of_source3 = 'ТЕКСТ'
                    link_to_source3 = batch[2]
                    batch_3 = batch[3]

            formatted_rag_prompt = rag_prompt.format(
                type_of_source1=type_of_source1,
                link_to_source1=link_to_source1,
                batch_1=batch_1,
                type_of_source2=type_of_source2,
                link_to_source2=link_to_source2,
                batch_2=batch_2,
                type_of_source3=type_of_source3,
                link_to_source3=link_to_source3,
                batch_3=batch_3,
                QUESTION=question
            )

            answer1 = message_llm(system_prompt, formatted_rag_prompt)
            answer2 = message_llm(system_prompt, formatted_rag_prompt)
            answer3 = message_llm(system_prompt, formatted_rag_prompt)

            formatted_som_prompt = sc_prompt.format(answer1=answer1,answer2=answer2,answer3=answer3)
            final_answer = message_llm(system_prompt, formatted_som_prompt)

            return f"{final_answer}\nИсточники: \n{link_to_source1}, \n{link_to_source2}, \n{link_to_source3}"
    except Exception as e:
        return f"Пожалуйста, посетите FAQ страницу сайта ЦБ РФ (https://www.cbr.ru/faq/) для получения ответа на этот вопрос."  


iface = gr.Interface(
    fn=ask_chatbot,
    inputs=[
        gr.Dropdown(choices=["zero-shot", "few-shot", "self-consistency"], label="Выберите режим"),
        gr.Textbox(lines=2, placeholder="Введите ваш вопрос здесь...", label="Вопрос")
    ],
    outputs=gr.Textbox(label="Ответ"),
    title="Чат с виртуальным помощником - SCOOF DEVELOPERS",
    description="Выберите режим и введите ваше сообщение, чтобы получить ответ от модели:"
)

# Запуск интерфейса
iface.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/gblssroman/anaconda3/lib/python3.11/site-packages/gradio/queueing.py", line 501, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/gblssroman/anaconda3/lib/python3.11/site-packages/gradio/route_utils.py", line 253, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/gblssroman/anaconda3/lib/python3.11/site-packages/gradio/blocks.py", line 1695, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/gblssroman/anaconda3/lib/python3.11/site-packages/gradio/blocks.py", line 1235, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/gblssroman/anaconda3/lib/python3.11/site-packages/anyio/to_thread.py", line 28, in run_sync
    return await get_asynclib().run